# CUDA编程模型---CUDA存储单元的使用与错误检测

#### 通过向量加法，我们已经学会了如何调用线程。接下来，我们来实践一下，如何利用Cuda处理矩阵。今天的课程将会介绍：
1. 二维矩阵的乘法
2. 如何分配线程和GPU存储单元
3. Cuda编程模型中的错误检测
4. Cuda编程模型中的事件

#### 矩阵乘法是科学计算和深度学习领域常见的操作，我们先来看一看CPU代码如何处理矩阵乘法

    void cpu_matrix_mult(int *h_a, int *h_b, int *h_result, int m, int n, int k) 
    {
        for (int i = 0; i < m; ++i) 
        {
            for (int j = 0; j < k; ++j) 
            {
                int tmp = 0.0;
                for (int h = 0; h < n; ++h) 
                {
                    tmp += h_a[i * n + h] * h_b[h * k + j];
                }
                h_result[i * k + j] = tmp;
            }
        }
    }

#### 这时，我们看到在CPU代码中，需要嵌套三个for循环，也就是说CPU的线程会一个接一个的求结果矩阵中的每一个数值，直到处理完所有数值。那么，我们在GPU中就可以申请很多个线程，每个线程来求结果矩阵中的一个数值，并同时完成。![matrix_mul](matrix_mul.png)

那么，首先我们要得到每一个执行线程，在Grid所有线程中的(x,y)坐标，如下图所示，即(Thread_x, Thread_y) 
![matrix_mul2](matrix_mul2.png)

![array_2to1.png](array_2to1.png)

也就是说，以上面的CPU代码为例，我们要让编号为(Thread_x, Thread_y)的线程读取a矩阵中的一行和b矩阵中的一列，然后把对应元素乘积并累加。

接下来，我们在[matrix_mul.cu](matrix_mul.cu)中完成这一过程，如果遇到麻烦，请参考[result1.cu](result1.cu)

修改Makefile文件，并编译

In [ ]:
!make

执行，并查看结果是否正确

In [ ]:
!./matrix_mul

利用nvprof来查看程序性能

In [ ]:
!sudo /usr/local/cuda/bin/nvprof ./matrix_mul

修改矩阵大小为1000*1000，并查看效果

In [ ]:
!make

In [ ]:
!sudo /usr/local/cuda/bin/nvprof ./matrix_mul

#### Cuda编程模型中的错误检测

可以查看Cuda error的四个函数：  
```C++
__host__​__device__​const char* 	cudaGetErrorName ( cudaError_t error )
Returns the string representation of an error code enum name.  

__host__​__device__​const char* 	cudaGetErrorString ( cudaError_t error )
Returns the description string for an error code.  

__host__​__device__​cudaError_t 	cudaGetLastError ( void )
Returns the last error from a runtime call.  

__host__​__device__​cudaError_t 	cudaPeekAtLastError ( void )
Returns the last error from a runtime call.  
```

这里我们采用第二个，并将其封装在error.cuh文件中：
```C++
#pragma once
#include <stdio.h>

#define CHECK(call)                                   \
do                                                    \
{                                                     \
    const cudaError_t error_code = call;              \
    if (error_code != cudaSuccess)                    \
    {                                                 \
        printf("CUDA Error:\n");                      \
        printf("    File:       %s\n", __FILE__);     \
        printf("    Line:       %d\n", __LINE__);     \
        printf("    Error code: %d\n", error_code);   \
        printf("    Error text: %s\n",                \
            cudaGetErrorString(error_code));          \
        exit(1);                                      \
    }                                                 \
} while (0)
```

那么我们就可以在代码中这么使用：
```C++
    CHECK(cudaMemcpy(d_b, h_b, sizeof(int)*n*k, cudaMemcpyHostToDevice));
```

接下来，大家在之前做的matrix_mul.cu文件，添加CHECK()，如果遇到麻烦，请参考[result2.cu](result2.cu)   
修改Makefile文件，编译程序

In [ ]:
!make

执行查看效果

In [ ]:
!./matrix_mul

修改
```C++
CHECK(cudaMemcpy(h_c, d_c, (sizeof(int)*m*k), cudaMemcpyDeviceToHost)); 
```
成
```C++
CHECK(cudaMemcpy(h_c, d_c, (sizeof(int)*m*k*2), cudaMemcpyDeviceToHost));
```

再编译一下，并执行查看结果

In [ ]:
!make

In [ ]:
!./matrix_mul

In [ ]:
!sudo /usr/local/cuda/bin/nvprof ./matrix_mul

这时我们就精准的定位了出现错误的文件，位置，以及错误原因

#### 2. Cuda编程模型中的事件。事件的本质就是一个标记，它与其所在的流内的特定点相关联。可以使用时间来执行以下两个基本任务：
- 同步流执行
- 监控设备的进展

流中的任意点都可以通过API插入事件以及查询事件完成的函数，只有事件所在流中其之前的操作都完成后才能触发事件完成。默认流中设置事件，那么其前面的所有操作都完成时，事件才出发完成。
事件就像一个个路标，其本身不执行什么功能，就像我们最原始测试c语言程序的时候插入的无数多个printf一样。

创建和销毁： 

声明:
```C++
cudaEvent_t event;
```
创建：
```C++
cudaError_t cudaEventCreate(cudaEvent_t* event);
```
销毁：
```C++
cudaError_t cudaEventDestroy(cudaEvent_t event);
```
添加事件到当前执行流：
```C++
cudaError_t cudaEventRecord(cudaEvent_t event, cudaStream_t stream = 0);
```
等待事件完成，设立flag：
```C++
cudaError_t cudaEventSynchronize(cudaEvent_t event);//阻塞
cudaError_t cudaEventQuery(cudaEvent_t event);//非阻塞
```
当然，我们也可以用它来记录执行的事件：
```C++
cudaError_t cudaEventElapsedTime(float* ms, cudaEvent_t start, cudaEvent_t stop);
```

接下来，我们就修改matrix_mul.cu程序，来测试一下核函数执行的时间，如果遇到麻烦，请参考[result3.cu](result3.cu)

编译并执行程序

In [ ]:
!make

In [ ]:
!./matrix_mul

In [ ]:
!sudo /usr/local/cuda/bin/nvprof ./matrix_mul

课后作业：
1. 当我们能申请的线程数很少，远远不够的时候怎么办？